In [ ]:
import pickle
import pandas as pd

# Carregando o modelo
with open('../model/naive_bayes_model.pkl', 'rb') as f:
    modelo = pickle.load(f)

# Colunas utilizadas no treinamento
colunas_modelo = ['REC_PFPJ', 'PAG_REGIAO', 'REC_REGIAO', 
                  'PAG_IDADE', 'REC_IDADE','NATUREZA', 
                  'VALOR', 'QUANTIDADE', 'TICKET_MEDIO']

In [ ]:
# Mapas baseados no LabelEncoder para as variáveis categóricas
mapeamento_rec_pfpj = {'Nao disponivel': 0, 'PF': 1, 'PJ': 2}
mapeamento_pag_regiao = {
    'CENTRO-OESTE': 0, 'NORDESTE': 1, 'NORTE': 2,
    'Nao informado': 3, 'SUDESTE': 4, 'SUL': 5
}
mapeamento_rec_regiao = mapeamento_pag_regiao.copy()
mapeamento_pag_idade = {
    'Nao informado': 0, 'Nao se aplica': 1,
    'até 19 anos': 2, 'entre 20 e 29 anos': 3, 'entre 30 e 39 anos': 4,
    'entre 40 e 49 anos': 5, 'entre 50 e 59 anos': 6, 'mais de 60 anos': 7
}
mapeamento_rec_idade = mapeamento_pag_idade.copy()
mapeamento_natureza = {
    'B2B': 0, 'B2G': 1, 'B2P': 2, 'G2B': 3, 'G2G': 4,
    'G2P': 5, 'Nao disponivel': 6, 'P2B': 7, 'P2G': 8, 'P2P': 9
}

# Função para previsão de PAG_PFPJ (output)
def prever_pag_pfpj(
    REC_PFPJ, PAG_REGIAO, REC_REGIAO, PAG_IDADE, REC_IDADE,
    NATUREZA, VALOR, QUANTIDADE, TICKET_MEDIO
):
    # Validação simples (pode ser expandida)
    if (REC_PFPJ not in mapeamento_rec_pfpj or
        PAG_REGIAO not in mapeamento_pag_regiao or
        REC_REGIAO not in mapeamento_rec_regiao or
        PAG_IDADE not in mapeamento_pag_idade or
        REC_IDADE not in mapeamento_rec_idade or
        NATUREZA not in mapeamento_natureza):
        return "Erro: selecione opções válidas."

    dados = pd.DataFrame([[
        mapeamento_rec_pfpj[REC_PFPJ],
        mapeamento_pag_regiao[PAG_REGIAO],
        mapeamento_rec_regiao[REC_REGIAO],
        mapeamento_pag_idade[PAG_IDADE],
        mapeamento_rec_idade[REC_IDADE],
        mapeamento_natureza[NATUREZA],
        VALOR,
        QUANTIDADE,
        TICKET_MEDIO
    ]], columns=colunas_modelo)

    pred = modelo.predict(dados)[0]

    mapping_saida = {0: 'Nao disponivel', 1: 'PF', 2: 'PJ'}
    return f"Previsão de PAG_PFPJ: {mapping_saida.get(pred, pred)}"

In [ ]:
import gradio as gr

# Opções para dropdowns, incluindo 'Nao disponivel' e 'Nao se aplica' etc
opcoes_rec_pfpj = list(mapeamento_rec_pfpj.keys())
opcoes_regiao = list(mapeamento_pag_regiao.keys())
opcoes_idade = list(mapeamento_pag_idade.keys())
opcoes_natureza = list(mapeamento_natureza.keys())

with gr.Blocks() as app:
    gr.Markdown("### Previsão da Classe PAG_PFPJ")

    with gr.Row():
        REC_PFPJ = gr.Dropdown(opcoes_rec_pfpj, label="REC_PFPJ")
        PAG_REGIAO = gr.Dropdown(opcoes_regiao, label="PAG_REGIAO")
        REC_REGIAO = gr.Dropdown(opcoes_regiao, label="REC_REGIAO")

    with gr.Row():
        PAG_IDADE = gr.Dropdown(opcoes_idade, label="PAG_IDADE")
        REC_IDADE = gr.Dropdown(opcoes_idade, label="REC_IDADE")
        NATUREZA = gr.Dropdown(opcoes_natureza, label="NATUREZA")

    with gr.Row():
        VALOR = gr.Number(label="VALOR")
        QUANTIDADE = gr.Number(label="QUANTIDADE")
        TICKET_MEDIO = gr.Number(label="TICKET_MEDIO")

    botao = gr.Button("Prever")
    saida = gr.Textbox(label="Resultado")

    botao.click(
        fn=prever_pag_pfpj,
        inputs=[REC_PFPJ, PAG_REGIAO, REC_REGIAO, PAG_IDADE, REC_IDADE,
                NATUREZA, VALOR, QUANTIDADE, TICKET_MEDIO],
        outputs=saida
    )

app.launch(share=True)